In [1]:
import json
import argparse
import ast
import argparse
import random
from datetime import date, timedelta

# Set seed and date boundaries
random.seed(42)


def save_jsonl(data, filename):
    with open(filename, "w+", encoding="utf-8") as file:
        for line in data:
            file.write(json.dumps(line) + "\n")

with open("../../Dataset_Helping/names.txt", "r") as file:
    names = ' '.join([line.strip().replace(" ", "") for line in file.readlines()])
    names_list = list(set(ast.literal_eval(names)))


forum_topics = []
with open("../../Dataset_Helping/A_Multi/A_Multi_forum.jsonl", "r") as file:
    for line in file:
        forum_topic = json.loads(line.strip())
        forum_topics.append(forum_topic)

In [3]:
def find_valid_list_prices(num_items=30, max_tries=10, max_price=3000):

    for _ in range(20):
        base_price_1 = list(range(0,num_items,2))
        random.shuffle(base_price_1)
        
        prices = []
        k_list = []
        l_list = []
        for i in base_price_1:
            k_min = 100*(i+1) - 50
            k_max = 100*(i+1) + 50
            
            for k in range(k_min, k_max, 10):
                
                if k > (max_price/2):
                    found = False
                    for _ in range(600):
                        l = (random.randint(100,  k) // 10) * 10
                        k_div_l = k / l
                        l_div_k = l / k
                        if (round(k_div_l, 2) == k / l) and (round(l_div_k, 2) == l / k) and k_div_l < 5 and l_div_k < 6 and l != k and l not in k_list:
                            if (k,l) not in prices and (l,k) not in prices:
                                l_list.append(l)    
                                k_list.append(k)
                                prices.append((k,l))
                                prices.append((l,k))
                                found = True
                                break
                    if found == True:
                        break
                if k < (max_price/2):
                    found = False
                    for _ in range(600):
                        l = (random.randint(k, max_price) // 10) * 10
                        l_div_k = l / k
                        k_div_l = k / l
                        if (round(l_div_k, 2) ==  l / k) and (round(k_div_l, 2) == k / l) and l_div_k < 3 and k_div_l < 3 and l != k and l not in k_list:
                            if (k,l) not in prices and (l,k) not in prices:
                                l_list.append(l)
                                k_list.append(k)
                                prices.append((k,l))
                                prices.append((l,k))
                                found = True
                                break
                    if found == True:
                        break

        # Assert no duplicates between l_list and k_list
        for l in l_list:
            if l in k_list:
                raise Exception(f"Found duplicate value {l} in both l_list and k_list")
        if len(prices) == num_items:
            random.shuffle(prices)
            list_price_1 = [item_1 for item_1, item_2 in prices]
            list_price_2 = [item_2 for item_1, item_2 in prices]
            return list_price_1, list_price_2
        
    print(len(list_price_1), len(list_price_2))
    raise Exception("Failed to generate valid list prices")

In [4]:


def generate_message_prices(list_price_1 , list_price_2, names, names_2, brands, topic, question, year=2024):
    message_prices = []
    random.shuffle(brands)
    item_to_buy = topic
    for idx in range(len(list_price_1)):
        brand = brands[idx]
        price = list_price_1[idx]
        low_price_model = random.randint(2013,2022)
        high_price_model = random.randint(low_price_model+1,2024)
        if list_price_1[idx] > list_price_2[idx]: # bought at high price
            mul = list_price_1[idx] / list_price_2[idx]
            model = high_price_model
            if mul < 2:
                if mul < 1:
                    raise Exception("mul is less than 1")
                mul = ((list_price_1[idx] - list_price_2[idx]) / list_price_2[idx]) * 100
                mul = round(mul, 2)
                if int(mul) == mul:
                    mul = int(mul)

                        # f"{item_to_buy} from {brand} that are older models with lower price are {list_price_2[idx]} dollars"
                bought = [f"{item_to_buy} from {brand} model {low_price_model}: {list_price_2[idx]} dollars", 
                        # f"The higher priced {item_to_buy} from {brand} is {mul} percents more expensive than the lower priced {item_to_buy} from {brand}"
                        f"{brand}, model {high_price_model}: {mul} percents more expensive than model {low_price_model}",
                        # f"I bought the higher priced model of {item_to_buy} from {brand}"
                        f"model {high_price_model} was purchased"]
            else:
                mul = round(mul, 2)
                if int(mul) == mul:
                    mul = int(mul)
                        # f"{item_to_buy} from {brand} that are older models with lower price are {list_price_2[idx]} dollars",
                bought = [f"{item_to_buy} from {brand} model {low_price_model}: {list_price_2[idx]} dollars",
                        # f"The higher priced {item_to_buy} from {brand} is {mul} times more expensive than the lower priced {item_to_buy} from {brand}",
                          f"{brand}, model {high_price_model}: {mul} times more expensive than model {low_price_model}",
                        # f"I bought the higher priced model of {item_to_buy} from {brand}")
                          f"model {high_price_model} was purchased"]
            high_price_brand = ("high_priced_brand", list_price_1[idx])
            low_price_brand = ("low_priced_brand", list_price_2[idx])
            
        else: # bought at low price
            mul = list_price_2[idx] / list_price_1[idx]
            model = low_price_model
            if mul < 2:
                if mul < 1:
                    raise Exception("mul is less than 1")
                mul = ((list_price_2[idx] - list_price_1[idx]) / list_price_2[idx]) * 100
                mul = round(mul, 2)
                if int(mul) == mul:
                    mul = int(mul)
                        # f"{item_to_buy} from {brand} that are newer models with higher price are {list_price_2[idx]} dollars",
                bought = [f"{item_to_buy} from {brand} model {high_price_model}: {list_price_2[idx]} dollars",
                        # f"The lower priced {item_to_buy} from {brand} is {mul} percents less expensive than the higher priced {item_to_buy} from {brand}",
                          f"{brand}, model {low_price_model}: {mul} percents less expensive than model {high_price_model}",
                        # f"I bought the lower priced model of {item_to_buy} from {brand}")
                          f"model {low_price_model} was purchased"]
            else:
                mul = round(mul, 2)
                if int(mul) == mul:
                    mul = int(mul)
                        # f"{item_to_buy} from {brand} that are newer models with higher price are {list_price_2[idx]} dollars",
                bought = [f"{item_to_buy} from {brand} model {high_price_model}: {list_price_2[idx]} dollars",
                        # f"The lower priced {item_to_buy} from {brand} is {mul} times less expensive than the higher priced {item_to_buy} from {brand}",
                          f"{brand}, model {low_price_model}: {mul} times less expensive than model {high_price_model}",
                        # f"I bought the lower priced model of {item_to_buy} from {brand}")
                          f"model {low_price_model} was purchased"]
                
                
            high_price_brand = ("high_priced_brand", list_price_1[idx])
            low_price_brand = ("low_priced_brand", list_price_2[idx])

        random_date = f"{year}-{random.randint(1,12)}-{random.randint(1,28)}"
        message_prices.append({"forum_post": (random_date, names[idx], bought),
                         "question": question.format(price=price),
                         "answer": (brand, model)})
    return message_prices

In [5]:

# ---------------------------
# Dataset Generation for Forum-Style Multi-Conversation
# ---------------------------
# Define overall interval for date generation.

# We'll generate dataset records for each topic.
# For each topic, we generate 20 (work_date, message_date) pairs.
# Then, for each pair, we assign a unique user (from a pool) and a forum item (randomly chosen).
dataset = []
shuffled_names_list = names_list.copy()
num_items = 20
max_price = 3000
# We'll generate enough unique users for each topic; here, 20 per topic.
for topic in forum_topics:
    dataset_row = {
        'topic': topic["topic"],
        'forum_question': topic["base_question"],
    }

    # Generate 20 item prices.
    list_price_1 , list_price_2 =  find_valid_list_prices(num_items, max_price) # select 30 prices for each user

    # Generate corresponding message dates.
    # Select 20 random names and remove them from shuffled_names_list
    topic_users = random.sample(shuffled_names_list, 20)
    for user in topic_users:
        shuffled_names_list.remove(user)
    topic_users_2 = random.sample(shuffled_names_list, 20)

    message_dates = generate_message_prices(list_price_1 , list_price_2, topic_users, topic_users_2, topic['items'], topic["topic"], topic['question'])
    dataset_row["posts"] = message_dates


    dataset.append(dataset_row)

# For demonstration, print the first 5 records.
for rec in dataset[:5]:
    print(rec)

1450 2900 False
650 1300 False
460 920 False
1650 660 False
1050 2100 False
1250 2500 False
1850 740 False
860 1720 False
50 100 False
250 500 False
250 500 False
1250 2500 False
460 1150 False
50 100 False
1050 2100 False
1650 660 False
850 1700 False
1450 2900 False
650 1300 False
1850 1480 False
1650 1320 False
850 1700 False
1250 2500 False
1850 740 False
650 1300 False
1050 2100 False
250 500 False
1450 2900 False
450 900 False
50 100 False
1250 2500 False
1050 2100 False
450 900 False
850 1700 False
1850 1480 False
1450 2900 False
1650 1320 False
250 500 False
650 1300 False
50 100 False
1250 2500 False
850 1700 False
250 500 False
50 100 False
1050 2100 False
650 1300 False
1850 1480 False
1450 2900 False
450 900 False
1650 660 False
650 1300 False
50 100 False
1050 2100 False
450 900 False
850 1700 False
1450 2900 False
1650 1320 False
1860 930 False
1250 2500 False
250 500 False
1650 1320 False
450 900 False
50 100 False
1050 2100 False
1850 740 False
1450 2900 False
250 500 F

In [13]:
import json
from pathlib import Path

# Create output directory if it doesn't exist
output_dir = Path("../../Dataset_Helping/A_Multi")
output_dir.mkdir(parents=True, exist_ok=True)

# Create filename with date range
output_file = output_dir / f"A_Multi_Structured.jsonl"

# Write data in jsonl format
with open(output_file, "w", encoding="utf-8") as f:
    for item in dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"Data saved to {output_file}")



Data saved to ../../Dataset_Helping/A_Multi/A_Multi_Structured.jsonl


## Merging Conversations and structured

In [3]:
import json 
# Load the structured data
structured_data_path = "../../Dataset_Helping/A_Multi/A_Multi_Structured.jsonl"

with open(structured_data_path, 'r', encoding='utf-8') as f:
    structured_data = [json.loads(line) for line in f]

print(f"Loaded {len(structured_data)} records from structured data file")

# Load the generated conversation data
generated_data_path = "../../Dataset_Helping/A_Multi/A_Multi_Structured_Generated_conversation.jsonl"

with open(generated_data_path, 'r', encoding='utf-8') as f:
    generated_data = [json.loads(line) for line in f]

print(f"Loaded {len(generated_data)} records from generated data file")



Loaded 25 records from structured data file
Loaded 500 records from generated data file


Struture:

topic - forum_question - forum_post(message_date) - forum_post(user) - forum_post(user_response) - question - answer

user_response is the conversation

In [4]:
dataset = []

x = 0
for i in range(len(structured_data)):
    topic = structured_data[i]['topic']
    forum_question = structured_data[i]['forum_question']
    posts = structured_data[i]['posts']
    for j in range(len(posts)):
        message_date = posts[j]['forum_post'][0]
        user = posts[j]['forum_post'][1]
        if i*20 + j != x:
            print(i*20 + j, x)
        assert i*20 + j == x
        x += 1
        user_response = generated_data[int(i*20 + j)]
        if user_response == '-':
            print(user_response, i*20 + j)
            raise Exception("user_response is '-'")
        dataset.append({
            "user_ID": i,
            "topic": topic,
            "forum_question": forum_question,
            "message_date": message_date,
            "user": user,
            "user_response": user_response,
            "question": posts[j]['question'],
            "answer": posts[j]['answer']
        })


In [5]:
with open('../../Data/A_Multi.jsonl', 'w', encoding='utf-8') as f:
    for item in dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [5]:
a = '''
 I read the To Kill a Mockingbird 6 days ago. The book had a significant impact on me, and I appreciated its thought-provoking themes. I found the characters to be well-developed and relatable, making the story even more engaging. Overall, I highly recommend To Kill a Mockingbird to anyone looking for a compelling and memorable read.
'''

['\n I read the To Kill a Mockingbird 6 days ago',
 ' The book had a significant impact on me, and I appreciated its thought-provoking themes',
 ' I found the characters to be well-developed and relatable, making the story even more engaging',
 ' Overall, I highly recommend To Kill a Mockingbird to anyone looking for a compelling and memorable read',
 '\n']